In [ ]:
# =====================================================
# FAKE NEWS DETECTION – ALBERT-base x FAKENEWSNET
# =====================================================

# 1. CÀI ĐẶT & IMPORT
!pip install -q transformers datasets torch scikit-learn pandas numpy psutil accelerate sentencepiece

import os, re, shutil, psutil, warnings
import pandas as pd
import numpy as np
import torch
from datasets import load_dataset, Dataset, DatasetDict, concatenate_datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    Trainer, TrainingArguments, EarlyStoppingCallback,
    DataCollatorWithPadding
)
from google.colab import drive

warnings.filterwarnings("ignore")

# 2. KIỂM TRA GPU
if torch.cuda.is_available():
    device_name = torch.cuda.get_device_name(0)
    print(f"Device: {device_name} | VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("Device: CPU")

# 3. MOUNT DRIVE & SETUP PATH
drive.mount('/content/drive', force_remount=False)
OUTPUT_DIR = "/content/drive/MyDrive/FakeNewsNet_ALBERT_Pro"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Hàm quản lý checkpoint (giữ ổ cứng không bị đầy)
def manage_checkpoints(output_dir, keep_latest=1):
    ckpts = [c for c in os.listdir(output_dir) if c.startswith("checkpoint-")]
    if len(ckpts) <= keep_latest: return
    # Sắp xếp theo thời gian sửa đổi
    ckpts_sorted = sorted(ckpts, key=lambda x: os.path.getmtime(os.path.join(output_dir, x)))
    for ck in ckpts_sorted[:-keep_latest]:
        shutil.rmtree(os.path.join(output_dir, ck), ignore_errors=True)
        print(f"🧹 Đã xóa checkpoint cũ: {ck}")

# 4. TẢI DATASET: FAKENEWSNET
print("\n⏳ Đang tải dataset FakeNewsNet...")
try:
    # Tải cả 2 miền dữ liệu
    ds_gossip = load_dataset("rickstello/FakeNewsNet", "gossipcop", split="train")
    ds_politi = load_dataset("rickstello/FakeNewsNet", "politifact", split="train")
    dataset_full = concatenate_datasets([ds_gossip, ds_politi])
    df = pd.DataFrame(dataset_full)
except Exception as e:
    print(f"⚠️ Tải config thất bại ({e}), thử tải default...")
    dataset = load_dataset("rickstello/FakeNewsNet", split="train")
    df = pd.DataFrame(dataset)

print(f"Tổng số mẫu: {len(df)}")

# 5. XỬ LÝ CỘT & LÀM SẠCH (Auto-detect columns)
# Tìm tên cột
text_col = next((c for c in ['news_content', 'text', 'content', 'body'] if c in df.columns), None)
title_col = next((c for c in ['title', 'news_title', 'headline'] if c in df.columns), None)
label_col = next((c for c in ['real', 'label', 'class', 'fake'] if c in df.columns), None)

print(f"Mapping: Text='{text_col}' | Title='{title_col}' | Label='{label_col}'")

if not label_col: raise ValueError("❌ Không tìm thấy cột nhãn!")

# Chuẩn hóa nhãn về cột 'label'
df['label'] = df[label_col]

# Ghép Title + Text an toàn
title_data = df[title_col].fillna('') if title_col else ""
text_data = df[text_col].fillna('') if text_col else ""
df['content'] = title_data + " [SEP] " + text_data

# Hàm làm sạch
def clean_text(s):
    if not isinstance(s, str): return ""
    s = s.lower()
    s = re.sub(r'https?://\S+', ' ', s)
    s = re.sub(r'<.*?>', ' ', s)
    s = re.sub(r'[^a-z0-9\s]', ' ', s)
    s = re.sub(r'\s+', ' ', s).strip()
    return s

print("🧹 Đang làm sạch văn bản...", end="")
df['content'] = df['content'].apply(clean_text)
df = df[df['content'].str.len() > 20].drop_duplicates(subset=['content'])
print(f" → Sau xử lý: {len(df):,}")

# Phân bố nhãn
print(f"Phân bố nhãn: {df['label'].value_counts(normalize=True).to_dict()}")

# Tính Class Weights (Cân bằng dữ liệu)
classes = np.unique(df['label'])
class_weights = compute_class_weight('balanced', classes=classes, y=df['label'])
class_weight_dict = {k: float(v) for k, v in zip(classes, class_weights)}
print("Class weights:", class_weight_dict)

# 6. SPLIT DATA
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['label'])

dataset_dict = DatasetDict({
    "train": Dataset.from_pandas(train_df[['content','label']].reset_index(drop=True)),
    "validation": Dataset.from_pandas(val_df[['content','label']].reset_index(drop=True)),
    "test": Dataset.from_pandas(test_df[['content','label']].reset_index(drop=True))
})

# 7. TOKENIZER (ALBERT)
MODEL_NAME = "albert-base-v2"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_fn(batch):
    return tokenizer(batch["content"], truncation=True, max_length=384, padding=False)

print("⚙️ Đang tokenize...")
tokenized = dataset_dict.map(tokenize_fn, batched=True, batch_size=1000, remove_columns=['content'])
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 8. MODEL SETUP
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)

# Cấu hình nhãn (FakeNewsNet: 1=Real, 0=Fake hoặc ngược lại, code tự map theo dữ liệu)
# Giả định cột 'real': 1 là Real, 0 là Fake
model.config.id2label = {0: "Fake", 1: "Real"}
model.config.label2id = {"Fake": 0, "Real": 1}

# 9. CUSTOM TRAINER (Weighted Loss)
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        # Truyền weight vào loss function
        w = torch.tensor(list(class_weight_dict.values()), dtype=torch.float32, device=model.device)
        loss_fct = torch.nn.CrossEntropyLoss(weight=w)
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

# 10. METRICS
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted", zero_division=0)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

# 11. TRAINING ARGUMENTS
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=5,                 
    per_device_train_batch_size=16,     
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=1,
    learning_rate=2e-5,                 
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    fp16=torch.cuda.is_available(),
    report_to="none"
)

trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

# 12. RUN TRAIN
print("\n🚀 BẮT ĐẦU HUẤN LUYỆN ALBERT...")
trainer.train()

# 13. EVALUATE & SAVE
print("\n🎯 ĐÁNH GIÁ TRÊN TEST SET:")
results = trainer.evaluate(tokenized["test"])
print(results)

# Lưu model cuối cùng
final_path = os.path.join(OUTPUT_DIR, "final_albert_fnn")
trainer.save_model(final_path)
tokenizer.save_pretrained(final_path)

# Dọn dẹp checkpoint thừa
manage_checkpoints(OUTPUT_DIR, keep_latest=0) # Xóa hết checkpoint, chỉ giữ model final
print(f"\n✅ Đã lưu model tại: {final_path}")